# Pasos de ETL
### 1) Lectura de datos de Yelp


***Lectura de librerías***

In [25]:
import pandas as pd

### 3.1) Lectura de csv Limpio de Google

In [26]:
datosGoogleMaps = pd.read_csv('datos_googleMaps.csv')
datosGoogleMaps.sample(2)

,rating,text,gmap_id,name,address,latitude,longitude,category,avg_rating,num_of_reviews
19051,5,NaN,0x89de15479581db75:0x7317e54966661ef2,CoreLife Eatery,"CoreLife Eatery, 11 Clifton Country Rd, Clifto...",42.859229,-73.781294,Restaurant,4.3,358
15702,2,I would not recommend this restuarant if you a...,0x89d30738ff11b1bb:0xa2c2f3a0f74f8774,The Dog Bar,"The Dog Bar, 1913 Davis Rd, West Falls, NY 14170",42.703273,-78.681934,Restaurant,4.4,108


### 3.2) Lectura de business.pkl

In [27]:
# lectura de archivos business
dfBusiness = pd.read_pickle('business.pkl')

# Buscar nulos en columnas
nulos_por_columnaBusiness = dfBusiness.isnull().sum()

#Eliminar columans innecesarias
dfBusiness = dfBusiness.iloc[:, :14]
dfBusiness = dfBusiness.drop(['is_open'], axis=1)
# Filtrar Data por Los estados a Trabajar 'NY' O 'FL' O 'CA'
dfBusiness = dfBusiness[dfBusiness['state'].isin(['NY', 'FL', 'CA'])]

dfBusiness["hours"]=dfBusiness["hours"].replace('{', '[', regex=True)
dfBusiness["hours"]=dfBusiness["hours"].replace('}', ']', regex=True)
dfBusiness["attributes"]=dfBusiness["attributes"].replace('"{', '[', regex=True)
dfBusiness["attributes"]=dfBusiness["attributes"].replace('}"', ']', regex=True)
dfBusiness["attributes"]=dfBusiness["attributes"].replace('{', '[', regex=True)
dfBusiness["attributes"]=dfBusiness["attributes"].replace('}', ']', regex=True)

# Filtro por restarurante 
dummies = dfBusiness['categories'].str.get_dummies(',')

# Concatena las columnas de dummies al DataFrame original
dfBusinessDummies = pd.concat([dfBusiness, dummies], axis=1)

# Elimina la columna original 'categories' si es necesario
dfBusinessDummies= dfBusinessDummies.drop(columns=['categories'])
# Quitar espacios innecesarios a columnas
dfBusinessDummies.columns = dfBusinessDummies.columns.str.strip()
# Filtro por Comidas
businessColumnas_categorias = ['Acai Bowls', 'Asian Fusion', 'Barbeque', 
                               'Buffets', 'Burgers', 'Cafes', 'Cafeteria', 
                               'Cajun/Creole', 'Chicken Wings', 'Child Care & Day Care', 
                               'Coffee & Tea', 'Coffee & Tea Supplies', 'Coffee Roasteries', 
                               'Comfort Food', 'Comic Books', 'Donuts', 'Empanadas', 'Fast Food', 
                               'Food', 'Food Court', 'Food Delivery Services', 'Food Stands', 'Food Tours', 
                               'Food Trucks', 'Juice Bars & Smoothies', 'Junk Removal & Hauling', 'Lounges', 
                               'Pizza', 'Pubs', 'Restaurants', 'Salad', 'Sandblasting', 'Sandwiches', 
                               'Seafood', 'Specialty Food', 'Steakhouses', 'Tacos', 'Vegan', 'Vegetarian']
# Filtrar las filas donde al menos una de las columnas de categorías tiene un valor de 1
dfBusinessDummies = dfBusinessDummies[dfBusinessDummies[businessColumnas_categorias].any(axis=1)]


columnasSelecBusiness = ['business_id', 'name','address','city','state','postal_code','postal_code','latitude','longitude','stars','review_count']
dfBusinessTotal  =dfBusinessDummies[columnasSelecBusiness]
dfBusinessTotal.sample(2)

,business_id,name,address,city,state,postal_code,postal_code,latitude,longitude,stars,review_count
44333,5O0-XVq49udK5s6n19JL0w,Piranha's Bar and Grill,248 Sanders Ferry Rd,Hendersonville,FL,37075,37075,36.285665,-86.608492,3.0,21
42356,ECmPXOHont5XlHp3zs4g4A,Spa Atlantis,"Atlantis Casino Resort Spa, 3800 S Virginia St",Reno,FL,89502,89502,39.48895,-119.793719,4.0,221


### 3.2.1 Exportar la tabla dfBusinessDummies para Modelo ML1

In [28]:
# Exportar Para modelo de Predicción
columnasModeloML1 = businessColumnas_categorias + ['business_id', 'name','address','city','state']
dfBusinessDummies = dfBusinessDummies[columnasModeloML1]
# Eliminar columnas Duplicadas
dfBusinessDummies_sin_duplicados = dfBusinessDummies.loc[:,~dfBusinessDummies.columns.duplicated()]
# Exportar
dataBusinesML1 = 'dataBusinesML1.csv'
dfBusinessDummies_sin_duplicados.to_csv(dataBusinesML1, index=False)
dfBusinessDummies_sin_duplicados.sample(1)

,Acai Bowls,Asian Fusion,Barbeque,Buffets,Burgers,Cafes,Cafeteria,Cajun/Creole,Chicken Wings,Child Care & Day Care,...,Specialty Food,Steakhouses,Tacos,Vegan,Vegetarian,business_id,name,address,city,state
89847,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,n874MqUWsFiJ7NGxorBTUg,Napoleon Burger,4229 Main St,Philadelphia,FL


In [29]:
# Exportar
DfBusinnessnombre_archivo_csv = 'dfBusinessTotal.csv'
dfBusinessTotal.to_csv(DfBusinnessnombre_archivo_csv, index=False)

### 3.3) Lectura de checkin.json

In [30]:
# Lectura de Archivo JSON
rutaCheckIn = "checkin.json"
# Lee el archivo JSON
dfCheckIn= pd.read_json(rutaCheckIn, lines=True)
dfCheckIn
#No se usará el algoritmo de Checkin, porque no es parte de nuestro MODELO DE ML, ni del objetivo del proyecto
dfCheckIn['num_registros'] = dfCheckIn['date'].str.count(',') + 1
dfCheckIn= dfCheckIn.drop(columns=['date'])
dfCheckIn

,business_id,num_registros
0,---kPU91CF4Lq2-WlRu9Lw,11
1,--0iUa4sNDFiZFrAdIWhZQ,10
2,--30_8IhuyMHbSOcNWd6DQ,2
3,--7PUidqRWpRSpXebiyxTg,10
4,--7jw19RH9JKXgFohspgQw,26
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,67
131926,zznZqH9CiAznbkV6fXyHWA,1
131927,zzu6_r3DxBJuXcjnOYVdTw,23
131928,zzw66H6hVjXQEt0Js3Mo4A,2


In [31]:
# Exportar
dfCheckInRuta = 'dfCheckInLimpio.csv'
dfCheckIn.to_csv(dfCheckInRuta, index=False)

### 3.4) Lectura de tip.json

In [32]:
rutaTip= "tip.json"
# Lee el archivo JSON
dfTip= pd.read_json(rutaTip, lines=True)
dfTip['date'] = pd.to_datetime(dfTip['date']).dt.strftime('%d-%m-%Y')
dfTip

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,18-05-2012,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,05-02-2013,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,18-08-2013,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,27-06-2017,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,06-10-2012,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,11-09-2021,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,30-10-2021,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,05-11-2021,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,20-11-2021,0


In [33]:
# Exportar
dfTipRuta = 'dfTipRutaLimpio.csv'
dfTip.to_csv(dfTipRuta, index=False)

### 3.5) Lectura de review.json

In [34]:
rutaReview= "review.json"
dfReviewYelp = pd.read_json(rutaReview, lines=True, nrows=100000)
dfReviewYelp['date'] = pd.to_datetime(dfReviewYelp['date']).dt.strftime('%d-%m-%Y')
dfReviewYelp['user_id'] = dfReviewYelp['user_id'].str.strip()
dfReviewYelp['review_id'] = dfReviewYelp['review_id'].str.strip()
dfReviewYelp['votos'] = dfReviewYelp['useful'] + dfReviewYelp['funny'] + dfReviewYelp['cool']
dfReviewYelp = dfReviewYelp.drop(columns=['funny', 'useful', 'cool'])

In [35]:
dfReviewYelp.sample(3)

,review_id,user_id,business_id,stars,text,date,votos
40777,lHtzjHG33lFjkR7OYyOxWg,0HP9AuJ2u7JtI2KIXADLrQ,p9tB3Fiszn1HMLUE-97K4g,5,Took my daughter out for a daddy/daughter date...,20-12-2016,0
26583,rjsGMzwNm8-ZorkfANnB4Q,XYSDrIef7g4Gmp3lNFVO6A,1o-PkLbF5CkHimxVtd-nVg,4,After gassing up the body on Nathan's Famous C...,15-05-2008,0
65341,pfprezSnWciJ2O3weKHkVw,z5CiTbBZeeHL3_2t2-ux0w,Aphp-EdFass877c-sWp5eg,4,"Arriving to NJ to visit a friend, we were imme...",22-04-2015,2


### 3.6 Lectura de user.parquet

In [36]:
# Lectura
rutaUser = 'user.parquet'
dfUser = pd.read_parquet(rutaUser)

In [37]:
dfUser.sample(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
229673,r_e0RXL7w0Ja30vyZ3edHg,Peter,44,2014-09-25 20:41:49,17,1,4,,"s_woARUbMaMup5nQvJbFTA, Mn119MZis9ffBk1rPCTonA...",0,...,0,0,0,0,1,0,0,0,0,0
1234689,LDAoFbKh_EmX5QiPabgzZg,Kaci,23,2011-06-14 21:20:47,10,3,2,,"LS3yCTWfTqm1RXOLf2cMDA, lgyG3E4Y0nps0irdVtO5RA...",0,...,0,0,0,0,0,1,0,0,1,0


In [38]:
# Poner Cantidad de Votos Totales
dfUserTotal = dfUser
dfUserTotal['votos'] = dfUser['useful'] + dfUser['funny'] + dfUser['cool']
# Eliminar filas no importantes
dfUserTotal = dfUserTotal.drop(columns=['review_count', 'yelping_since', 'useful', 'funny','cool', 
                                             'elite', 'friends', 'average_stars', 
                                             'compliment_hot', 'compliment_more', 'compliment_profile', 
                                             'compliment_cute', 'compliment_list', 'compliment_note', 
                                             'compliment_plain', 'compliment_cool', 'compliment_funny', 
                                             'compliment_writer', 'compliment_photos'])
dfUserTotal['user_id'] = dfUserTotal['user_id'].str.strip()

In [39]:
dfUserTotal.sample(3)

,user_id,name,fans,votos
1823231,RA4bM_MOtPTJe4hBqfXTyA,Elle,0,6
868383,nSXSYEFlg8ORxJUbQ5NlYA,Chris,0,0
1651316,OdhjyG6Ti8QvpcsMejlehw,Arianna,0,14


***Uniones

In [40]:
# Exportar
dfUserTotalRuta = 'dfUserTotalLimpio.csv'
dfUserTotal.to_csv(dfUserTotalRuta, index=False)

In [41]:
# Merge entre Reviews y BusinessID
mergeReviewBusiness = dfReviewYelp.merge(dfBusinessTotal, on='business_id', how='inner')